In [1]:
# Dependencies
import requests
import json
import prettytable
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import linregress
import numpy as np
import csv
import geopandas as gp
from scipy.stats import f_oneway

In [2]:
# CEU6054151111
headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": ['CEU6054151111'],"startyear":"2013", "endyear":"2022"})
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)
for series in json_data['Results']['series']:
    x=prettytable.PrettyTable(["series id","year","period","value","footnotes"])
    seriesId = series['seriesID']
    for item in series['data']:
        year = item['year']
        period = item['period']
        value = item['value']
        footnotes=""
        for footnote in item['footnotes']:
            if footnote:
                footnotes = footnotes + footnote['text'] + ','
        if 'M01' <= period <= 'M12':
            x.add_row([seriesId,year,period,value,footnotes[0:-1]])
    output = open(seriesId + '.txt','w')
    output.write (x.get_string())
    output.close()
json_data

{'status': 'REQUEST_SUCCEEDED',
 'responseTime': 211,
 'message': [],
 'Results': {'series': [{'seriesID': 'CEU6054151111',
    'data': [{'year': '2022',
      'period': 'M12',
      'periodName': 'December',
      'value': '2107.88',
      'footnotes': [{}]},
     {'year': '2022',
      'period': 'M11',
      'periodName': 'November',
      'value': '2137.56',
      'footnotes': [{}]},
     {'year': '2022',
      'period': 'M10',
      'periodName': 'October',
      'value': '2275.90',
      'footnotes': [{}]},
     {'year': '2022',
      'period': 'M09',
      'periodName': 'September',
      'value': '2131.54',
      'footnotes': [{}]},
     {'year': '2022',
      'period': 'M08',
      'periodName': 'August',
      'value': '2101.40',
      'footnotes': [{}]},
     {'year': '2022',
      'period': 'M07',
      'periodName': 'July',
      'value': '2134.91',
      'footnotes': [{}]},
     {'year': '2022',
      'period': 'M06',
      'periodName': 'June',
      'value': '2098.28',
 

In [3]:
# Store the year, period, and periodName, and # of job openings in columns of the dataframe
year = []
periodName = []
value = []
num_series = len(json_data["Results"]["series"][0]["data"])
series_data = json_data["Results"]["series"][0]["data"]

for i in range(num_series):
    year.append(series_data[i]["year"])
    periodName.append(series_data[i]["periodName"])
    value.append(series_data[i]["value"])

# Create a dataframe from these columns
salary_df = pd.DataFrame({"Year": year,
                            "Month": periodName,
                            "Weekly Salary": value})


In [4]:
salary_df

,Year,Month,Weekly Salary
0,2022,December,2107.88
1,2022,November,2137.56
2,2022,October,2275.90
3,2022,September,2131.54
4,2022,August,2101.40
...,...,...,...
115,2013,May,1594.86
116,2013,April,1603.25
117,2013,March,1593.32
118,2013,February,1612.00


In [5]:
salary_df.dtypes

Year             object
Month            object
Weekly Salary    object
dtype: object

In [6]:
salary_df['Weekly Salary'] = salary_df['Weekly Salary'].astype(float)

In [7]:
salary_df.dtypes

Year              object
Month             object
Weekly Salary    float64
dtype: object

In [8]:
salary_df['Monthly Salary'] = salary_df['Weekly Salary'] * 4.23
salary_df

,Year,Month,Weekly Salary,Monthly Salary
0,2022,December,2107.88,8916.3324
1,2022,November,2137.56,9041.8788
2,2022,October,2275.90,9627.0570
3,2022,September,2131.54,9016.4142
4,2022,August,2101.40,8888.9220
...,...,...,...,...
115,2013,May,1594.86,6746.2578
116,2013,April,1603.25,6781.7475
117,2013,March,1593.32,6739.7436
118,2013,February,1612.00,6818.7600


In [9]:
salary_df = salary_df.groupby('Year').sum('Monthly Salary')
salary_df

,Weekly Salary,Monthly Salary
Year,,
2013,19375.38,81957.8574
2014,20028.60,84720.9780
2015,20976.82,88731.9486
2016,21242.66,89856.4518
2017,21807.35,92245.0905
2018,22651.19,95814.5337
2019,22962.73,97132.3479
2020,23252.71,98358.9633
2021,23763.87,100521.1701


In [10]:
salary_df = salary_df.rename(columns={"Monthly Salary": "Yearly Salary"})

In [11]:
salary_df = salary_df.drop(['Weekly Salary'], axis=1)

In [12]:
salary_df

,Yearly Salary
Year,
2013,81957.8574
2014,84720.9780
2015,88731.9486
2016,89856.4518
2017,92245.0905
2018,95814.5337
2019,97132.3479
2020,98358.9633
2021,100521.1701


In [13]:
salary_df.to_csv('BLS_annual_avg_salary.csv')